## Prereqs:

pip install autogen-agentchat==0.5.1  
pip install autogen-ext[gemini]==0.5.1  
pip install autogen-agentchat==0.5.1  

In [1]:
import os
os.chdir('./../')

In [2]:
import sqlite3
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_core.models import ModelFamily
from autogen_ext.models.openai import OpenAIChatCompletionClient

from common.constants import Constants

In [3]:
load_dotenv(override=True)

True

In [4]:
model_client = OpenAIChatCompletionClient(model=Constants.GEMINI_MODEL_LITE, 
                                          api_key=os.getenv(Constants.GOOGLE_API_KEY),
                                          model_info={'vision': True,
                                                      'function_calling': True,
                                                      'json_output': True,
                                                      'family': ModelFamily.GEMINI_2_0_FLASH,
                                                      'structured_output': True})

In [5]:
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, content="I'd like to go to London", type='TextMessage')

In [6]:
agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

In [7]:
response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Ah, London! Excellent choice. Just try not to lose your umbrella, it\'s practically a national treasure over there. We\'ll get you there faster than you can say "Mind the gap!"'

In [8]:
_DB_NAME_WITH_PATH = './autogen_05/tickets.db'

if os.path.exists(_DB_NAME_WITH_PATH):
    os.remove(_DB_NAME_WITH_PATH)

conn = sqlite3.connect(_DB_NAME_WITH_PATH)
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [9]:
def save_city_price(city_name, round_trip_price):
    with sqlite3.connect(_DB_NAME_WITH_PATH) as conn:
        c = conn.cursor()
        c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
        conn.commit()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [10]:
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    with sqlite3.connect(_DB_NAME_WITH_PATH) as conn:
        c = conn.cursor()
        c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
        result = c.fetchone()
    return result[0] if result else None

In [11]:
get_city_price("Rome")

499.0

In [12]:
smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [13]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='', is_error=False)]


"Sure, London it is! Get ready for some tea and crumpets. A roundtrip ticket will only set you back $299.0, which is basically a steal for all the royal wave-watching you'll be doing!"